# Preparation of data
This section will consist to exatct and transform data to be ready to upload in data warehouse


In [1]:
# Import all needed packages or modules
from pyspark.sql import *
from pyspark.sql.functions import *
from datetime import datetime, date
import pandas as pd

In [2]:
# Creation of spark Session
spark = SparkSession.builder.getOrCreate()

## 1. Extraction of data

In [3]:
# Import all data from your device or github. 
# In my case i downlad data in my device then i precise the path of the folder in my device
path1 ="E:/Learning/repos/Information_Data_Driven_Tokyo2020_Olympic/"

## Althletes data
url1 =path1+"data/raw/athletes.csv"
df0_athletes = spark.read.format("csv")\
                            .option("header",True)\
                            .load(url1)

## Coaches data
url2 =path1+"data/raw/coaches.csv"
df0_coaches = spark.read.format("csv")\
                            .option("header",True)\
                            .load(url2)

## Medals data
url3 =path1+"data/raw/medals.csv"
df0_medals = spark.read.format("csv")\
                            .option("header",True)\
                            .load(url3)

## country data
url4 =path1+"data/raw/all.csv"
df0_countries = spark.read.format("csv")\
                            .option("header",True)\
                            .load(url4)

We don't need all columns in each dataset and also to avoid redondance data in our warehouse we need to code somme data like names of althletes reapet in athletes data and medals also the discipline's name repeat in all 3 datasets.

In [13]:
## Columns need in Athletes dataset
df1_athletes = df0_athletes["name","gender","birth_date","country_code","discipline_code"]

## Columns need in Coaches dataset
df1_coaches = df0_coaches["name","gender","birth_date","country_code","discipline","function"]

## Columns need in Medals dataset
df1_medals = df0_medals["medal_code","medal_date","athlete_name","country_code","discipline_code","event"]

## Columns need in countries dataset
df1_countries = df0_countries["name","alpha-3","region","sub-region"]

In [5]:
## Structure of dataframe before convert
df1_athletes.printSchema()
df1_coaches.printSchema()
df1_medals.printSchema()

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline_code: string (nullable = true)

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- function: string (nullable = true)

root
 |-- medal_code: string (nullable = true)
 |-- medal_date: string (nullable = true)
 |-- athlete_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline_code: string (nullable = true)
 |-- event: string (nullable = true)



## 2. Tranformation of data

In [9]:
## convert birth_date from string to date
df1_athletes = df1_athletes.withColumn("birth_date",to_date(col("birth_date"),"yyyy-MM-dd"))
df1_coaches = df1_coaches.withColumn("birth_date",to_date(col("birth_date"),"yyyy-MM-dd"))
df1_medals = df1_medals.withColumn("medal_date",split(df1_medals["medal_date"]," ").getItem(0))\
                        .withColumn("medal_date",to_date(col("medal_date"),"yyyy-MM-dd"))

In [10]:
## Structure of dataframe after convert
df1_athletes.printSchema()
df1_coaches.printSchema()
df1_medals.printSchema()

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline_code: string (nullable = true)

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- function: string (nullable = true)

root
 |-- medal_code: string (nullable = true)
 |-- medal_date: date (nullable = true)
 |-- athlete_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline_code: string (nullable = true)
 |-- event: string (nullable = true)



In [20]:
## Renamed some columns for countries dataset
df1_countries= df1_countries.withColumnRenamed("alpha-3","country_code")\
                            .withColumnRenamed("region","continent")

In [21]:
df1_countries.show()

+-------------------+------------+---------+--------------------+
|               name|country_code|continent|          sub-region|
+-------------------+------------+---------+--------------------+
|        Afghanistan|         AFG|     Asia|       Southern Asia|
|      Åland Islands|         ALA|   Europe|     Northern Europe|
|            Albania|         ALB|   Europe|     Southern Europe|
|            Algeria|         DZA|   Africa|     Northern Africa|
|     American Samoa|         ASM|  Oceania|           Polynesia|
|            Andorra|         AND|   Europe|     Southern Europe|
|             Angola|         AGO|   Africa|  Sub-Saharan Africa|
|           Anguilla|         AIA| Americas|Latin America and...|
|         Antarctica|         ATA|     null|                null|
|Antigua and Barbuda|         ATG| Americas|Latin America and...|
|          Argentina|         ARG| Americas|Latin America and...|
|            Armenia|         ARM|     Asia|        Western Asia|
|         